Inferencing Gemma-2-2bit 

In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

modelName = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map="auto" , # Map all model layers to GPU 1,
    torch_dtype=torch.bfloat16
)

/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.08it/s]


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer=AutoTokenizer.from_pretrained(modelName)
# Example anti-vaccine comment
# input_text = "Generate counter argument for tweet: Another Covid 19 vaccine casualty Illaria Pappa 31 school teacher from #Italy died from thromboembolism 10 days after receiving #AstraZeneca #vaccine"
# input_ids = tokenizer(input_text, return_tensors="pt")
# input_ids = {key: value.to(device) for key, value in input_ids.items()}

# # print the input text
# print(input_text)
# # Generate the counter-argument
# outputs = model.generate(
#     **input_ids,
#     max_length=150,  # Increase the maximum length of the output
#     num_return_sequences=1,  # Generate one output
#     temperature=0.8,  # Control randomness (lower values = more deterministic)
#     top_p=0.9,  # Use nucleus sampling
#     top_k=50,  # Use top-k sampling
#     repetition_penalty=1.2,  # Penalize repeated phrases
#     do_sample=True  # Enable sampling for diverse outputs
# )

# # Decode the output
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Load Dataset

In [4]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_without_labels.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_CA_without_labels.csv',split='train')

Fine-Tune 

In [5]:
model_id = "google/gemma-2b-it"
model_name = "google/gemma-2b-it"

tokenizer_id = "google/gemma-2b-it"
tokenizer=AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.paddeing_side='right'

In [6]:
def create_message_column_without_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
#     print(row['text'])
    user = {
        "content": f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n##Output: ",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

Include Label Descriptions

In [6]:
labels_map = {}
labels_map['religious'] = "religious beliefs and their influence on views about vaccines"
labels_map['political'] = "the political factors that affect perceptions of vaccine use"
labels_map['ingredients'] = "concerns about the ingredients and chemical components in vaccines"
labels_map['unnecessary'] = "the importance and necessity of getting vaccinated to prevent diseases"
labels_map['conspiracy'] = "conspiracy theories suggesting hidden motives behind vaccination efforts"
labels_map['mandatory'] = "the debate over personal choice versus mandates in vaccination policies"
labels_map['ineffective'] = "evidence and reasons that support the effectiveness of vaccines"
labels_map['side-effect'] = "potential side effects and adverse reactions associated with vaccines"
labels_map['pharma'] = "the role of pharmaceutical companies and concerns about profit motives"
labels_map['rushed'] = "claims that vaccines were approved or developed without sufficient testing"
labels_map['country'] = "national biases and objections to vaccines produced by specific countries"

In [7]:
def create_message_column_with_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    # print(row)
    prompt= f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n Talk About "

    lab=row['labels'].split()
    if isinstance(lab, list):
            mapped_labels = " and ".join([labels_map.get(l, "") for l in lab])
    else:
            mapped_labels = labels_map.get(lab, "")
    # print(mapped_labels)
    prompt+= mapped_labels
    prompt+= " ##Output:"
    user = {
        "content": prompt,
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


In [8]:
# from datasets import load_dataset
# train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_with_label_desc.csv',split='train')
# val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_data_with_labels_desc.csv',split='train')

# train_data=train_data[:5]
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

In [7]:

dataset_chatml = train_data.map(create_message_column_without_labels)

dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_without_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

In [9]:
dataset_chatml[0]

{'ID': '1330212529956671491t',
 'text': '<bos><start_of_turn>user\nGenerate Counter Argument for the anti-vaccine tweet:\n Tweet: #CORRUPTION by pushing through #Pfizer vaccine which didn ’ t do a peer review & were able to push to the finish line & why ? Because , they are #BigPhama 2 many ? About the data ! @InovioPharma is transparent & submitted a peer review over 4 months ago & still no answer ? The news\n##Output:<end_of_turn>\n<start_of_turn>model\nWhile it is important to have transparency and peer review in the development of vaccines, it is also crucial to prioritize speed in a situation like a global pandemic. Pfizer may have pushed through their vaccine quickly because the need for a vaccine was urgent. Additionally, Pfizer has a long history of successful vaccine development and regulatory approval. Just because Inovio Pharma submitted a peer review earlier does not necessarily mean their vaccine is a better option. Fast-tracking the Pfizer vaccine could have been a necess

In [9]:

import torch
import peft ,trl

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
from trl import SFTTrainer

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# # This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [11]:
# tokenizer.pad_token_id=tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
# set cuda available device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

use_4bit = True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True
bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto",
#           attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model = prepare_model_for_kbit_training(model)
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
peft_config = LoraConfig(
        r=4,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)
# model = get_peft_model(model, peft_config)


Using GPU: NVIDIA RTX A5000


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


In [14]:

args = TrainingArguments(
        output_dir="/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/gemma-2b-bit",
        evaluation_strategy="epoch",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        learning_rate=1e-4,
        logging_dir="./logs",  # Directory for storing logs
        logging_steps=100,     # Log after every 100 steps
        logging_strategy="steps",  # Log by steps instead of epochs
        fp16=True,
        eval_steps=100,
        num_train_epochs=2,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        report_to= None,
        seed=42,
)



/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the 

In [14]:
print(torch.cuda.current_device())

0


In [15]:
import os
from rouge_score import rouge_scorer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
print(torch.cuda.current_device())
# import os
os.environ["WANDB_DISABLED"] = "true"
# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# make a data collator for the model
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
response_template = "<start_of_turn>model"
instruction_template = "<start_of_turn>user"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,response_template=response_template, tokenizer=tokenizer)


trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml,
        eval_dataset=val_dataset_chatml,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=args,
        data_collator=collator
)
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:2065: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value y

Epoch,Training Loss,Validation Loss
1,1.172500,1.112122
2,0.791300,1.113014


[codecarbon INFO @ 17:04:22] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 17:04:22] Energy consumed for all GPUs : 0.001106 kWh. Total GPU Power : 265.17841290687613 W
[codecarbon INFO @ 17:04:22] Energy consumed for all CPUs : 0.000226 kWh. Total CPU Power : 54.1770810097197 W
[codecarbon INFO @ 17:04:22] 0.001528 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:04:37] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 17:04:37] Energy consumed for all GPUs : 0.002255 kWh. Total GPU Power : 275.8743696777042 W
[codecarbon INFO @ 17:04:37] Energy consumed for all CPUs : 0.000452 kWh. Total CPU Power : 54.212400346801715 W
[codecarbon INFO @ 17:04:37] 0.003099 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:04:52] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 17:04:52] Energy consumed for all GPUs : 0.003403 kWh. Total

In [17]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


    PyYAML (>=5.1.*)
            ~~~~~~^


Evaluation

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
model_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/gemma-2b-bit"
tokenizer_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/gemma-2b-bit"
# connect to device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)



Using GPU: NVIDIA RTX A5000


loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--google--gemma-2b-it/snapshots/96988410cbdaeb8d5093d1ebdc5a8fb563e02bad/config.json
Model config GemmaConfig {
  "_name_or_path": "google/gemma-2b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0.dev0",
  "use_cache": true,
  "vocab_size": 256000
}

CUDA backend validation successful.
loading weights file model.safetensors from cache at /home/soh

In [18]:
!python -m pip install rouge_score

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [17]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

In [18]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [19]:
val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)
dataset_chatml = val_dataset_chatml.train_test_split(test_size=0.05)
dataset_chatml

Map: 100%|██████████| 987/987 [00:00<00:00, 5583.20 examples/s]


DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels', 'counter_argument', 'messages'],
        num_rows: 937
    })
    test: Dataset({
        features: ['ID', 'text', 'labels', 'counter_argument', 'messages'],
        num_rows: 50
    })
})

In [20]:
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2, max_time= 50)
    # print(outputs[0]['generated_text'])
    return outputs[0]['generated_text'][len(prompt):].strip()

In [32]:
dataset_chatml['test'][1]

{'ID': '1330967872718290945t',
 'text': '<bos><start_of_turn>user\nGenerate Counter Argument for the tweet:\n Tweet: Is there a legal loophole to avoid the covid 19 vaccine ? Asking for a friend .<end_of_turn>\n<start_of_turn>model\nWhile there may be some exemptions for medical or religious reasons, it is important to consider the greater good of public health. Avoiding the vaccine could put not only yourself at risk, but also those around you. It is important to follow public health guidelines and do your part to help stop the spread of COVID-19.<end_of_turn>\n',
 'labels': 'none',
 'counter_argument': 'While there may be some exemptions for medical or religious reasons, it is important to consider the greater good of public health. Avoiding the vaccine could put not only yourself at risk, but also those around you. It is important to follow public health guidelines and do your part to help stop the spread of COVID-19.',
 'messages': [{'content': 'Generate Counter Argument for the tw

[codecarbon INFO @ 18:37:22] Energy consumed for RAM : 0.012537 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 18:37:22] Energy consumed for all GPUs : 0.195854 kWh. Total GPU Power : 609.8920808754463 W
[codecarbon INFO @ 18:37:22] Energy consumed for all CPUs : 0.017640 kWh. Total CPU Power : 60.944878536275255 W
[codecarbon INFO @ 18:37:22] 0.226031 kWh of electricity used since the beginning.


In [25]:
test_inference(dataset_chatml['test'][1]['messages'][0]['content'])

'While it is true that some younger individuals may experience long-term effects from the vaccine, it is important to consider the overall benefits of getting vaccinated. Vaccines have been proven to be safe and effective in preventing the spread of deadly diseases. Additionally, the long-term effects of the vaccine are still being studied and understood, so it is not fair to assume that Pfizer would not have tested for any potential risks. It is also worth noting that the benefits of getting vaccinated, such as protecting oneself and others from serious illness, far outweigh the potential risks.\n souverainment, it is not fair to assume that Pfizer would not have tested for potential risks in the vaccine. Vaccines go through rigorous testing and approval processes to ensure their safety and efficacy. It is important to trust the expertise of healthcare professionals and public health officials who have thoroughly reviewed and approved the vaccine.\n\nWhile it is true that the long-ter

In [21]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [30]:
dataset_chatml['test'][1]['text']

'<bos><start_of_turn>user\nGenerate Counter Argument for the tweet:\n Tweet: Is there a legal loophole to avoid the covid 19 vaccine ? Asking for a friend .<end_of_turn>\n<start_of_turn>model\nWhile there may be some exemptions for medical or religious reasons, it is important to consider the greater good of public health. Avoiding the vaccine could put not only yourself at risk, but also those around you. It is important to follow public health guidelines and do your part to help stop the spread of COVID-19.<end_of_turn>\n'

In [22]:
num_samples=50
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=1, max_new_tokens=200, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2,
                 )
# print(outputs)
preds = []
for i in range(len(outputs)):
    generated_text = outputs[i][0]['generated_text']
    response = generated_text[len(prompts[i]):].split()
    # print(response)
    if len(response) > 1:
        # Extract the counter argument
        pred = " ".join(response)
    else:
        pred = ""  # Handle case with no valid split
    preds.append(pred)

    # Print prediction and corresponding reference and tweet 

    # print(f"Tweet {i + 1}: {dataset_chatml['test'][i]['text']}")
    print(f"Prediction {i + 1}: {pred}")
    print(f"Reference {i + 1}: {dataset_chatml['test'][i]['counter_argument']}")
    print("---")  # Separator for clarity
references= [dataset_chatml['test'][i]['counter_argument'] for i in range(len(outputs))]
metric.add_batch(predictions=preds, references=references)
tweets = [dataset_chatml['test'][i]['text'] for i in range(len(outputs))]

# save in a csv file
import pandas as pd
df = pd.DataFrame(list(zip(tweets, preds, references)), columns=['tweet', 'prediction', 'reference'])
df.to_csv("/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/gemma-2b-bit_finetuned_cross_2_labels.csv", index=False)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


Prediction 1: It's important to remember that the COVID-19 vaccines have been rigorously tested and approved by health authorities. Trusting in the science and data behind the vaccines is crucial in combating the spread of the virus and protecting public health. gră gră Actually, vaccines have been proven to be effective in preventing serious illness and death from COVID-19. Trusting in the scientific community and public health officials is essential in achieving herd immunity and ending the pandemic. gră gră
Reference 1: It's important to remember that vaccines undergo rigorous testing and approval processes before being distributed to the public. Trusting in the science behind the vaccine can help protect not only yourself, but also those around you.
---
Prediction 2: There is no legal loophole to avoid getting vaccinated against COVID-19. Getting vaccinated is a crucial step in protecting yourself and others from the virus. It is not possible to legally opt out of receiving the vac

In [23]:
metric.add_batch(predictions=preds, references=references)
result = metric.compute(use_stemmer=True)

In [24]:
import numpy as np

# Assuming result contains your ROUGE scores
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print("Rouge 1 Mean: ", rouge1_mean)
print("Rouge 2 Mean: ", rouge2_mean)
print("Rouge L Mean: ", rougeL_mean)
print("Rouge Lsum Mean: ", rougeLsum_mean)

Rouge 1 Mean:  41.43466509100617
Rouge 2 Mean:  16.98366220525522
Rouge L Mean:  27.516073247298234
Rouge Lsum Mean:  27.51243417756933


Evaluation Using Bert Scores

In [59]:
!pip install bert-score


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [25]:
from bert_score import score
P, R, F1 = score(preds, references, lang="en", verbose=True)

# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/vo

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  3.75it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 46.99it/s]

done in 0.57 seconds, 88.44 sentences/sec
BERTScore Precision: 0.8778289556503296
BERTScore Recall: 0.8938378691673279
BERTScore F1: 0.8856953382492065
